<a href="https://colab.research.google.com/github/WinstonMuijs/Natural-Language-Processing/blob/main/scikit_learn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 'Bouwen van woord-vectoren in scikit learn'



1.   verzamelen en preprocessen data
2.   bepalen label
3.   splitten van data in train en test
4.   Features van de tekst eruit halen in de vorm van een BOW-vector, als input, om voorspellingen te kunnen doen.
5.   evaluatie getraind model.



Importeren van Pandas en Scikit Learn modules

In [2]:
import pandas as pd

In [3]:
from sklearn.model_selection import train_test_split

In [4]:
from sklearn.feature_extraction.text import CountVectorizer

Creeren van een DataFrame op basis van een csv-file

In [18]:
df = pd.read_csv("fake_or_real_news.csv")
print(df)

      Unnamed: 0                                              title  \
0           8476                       You Can Smell Hillary’s Fear   
1          10294  Watch The Exact Moment Paul Ryan Committed Pol...   
2           3608        Kerry to go to Paris in gesture of sympathy   
3          10142  Bernie supporters on Twitter erupt in anger ag...   
4            875   The Battle of New York: Why This Primary Matters   
...          ...                                                ...   
6330        4490  State Department says it can't find emails fro...   
6331        8062  The ‘P’ in PBS Should Stand for ‘Plutocratic’ ...   
6332        8622  Anti-Trump Protesters Are Tools of the Oligarc...   
6333        4021  In Ethiopia, Obama seeks progress on peace, se...   
6334        4330  Jeb Bush Is Suddenly Attacking Trump. Here's W...   

                                                   text label  
0     Daniel Greenfield, a Shillman Journalism Fello...  FAKE  
1     Google Pinter

De labels vormen de ouput van het model. Het zijn de labels die je wilt dat het model levert of voorspelt. 

In [6]:
y = df.label

Je splitst je dataset in een train- en testset en je zegt dat de input-waarden van het model de tekstfeatures (rijen) zijn van de DataFrame, de output waarden y, en dat je de set verdeelt voor 33% in test en 66% in train data.

In [7]:
X_train, X_test , y_train, y_test = train_test_split(df['text'], y, test_size = 0.33, random_state = 53)

Tekst wordt omgezet in een Bag of Words vector, net als een corpus in gensim. Een corpus transformeert een document in een Bag of Words door token id's en de frequentie van de woorden - tokens - in het document te nemen en deze in een lijst met tuples weer te geven. [[(id, freq,), (id, freq) ,(id, freq) ]] 
CountVectorizer doet ongeveer hetzelfde en een bag of words vector, waarbij elke token een feature is.

In [8]:
count_vectorizer = CountVectorizer(stop_words = 'english')

Er wordt nu een BOW-vector gemaakt van de trainingsdata: mapping van de woorden met id's en vectoren hoeveel x een woord verschijnt in de tekst.

In [9]:
count_train = count_vectorizer.fit_transform(X_train.values)

Ook voor de test data

In [10]:
count_test = count_vectorizer.transform(X_test.values)

In [11]:
# Print the first 5 features of the count_vectorizer
print(count_vectorizer.get_feature_names_out()[:5])

['00' '000' '0000' '00000031' '000035']


TfidfVectorizer gebruikt ipv CountVectorizer

In [13]:
# Import TfidfVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

# Initialize a TfidfVectorizer object: tfidf_vectorizer
tfidf_vectorizer = TfidfVectorizer(stop_words='english', max_df=0.7)

# Transform the training data: tfidf_train 
tfidf_train = tfidf_vectorizer.fit_transform(X_train.values)

# Transform the test data: tfidf_test 
tfidf_test = tfidf_vectorizer.transform(X_test.values)

# Print the first 10 features
print(tfidf_vectorizer.get_feature_names_out()[:10])

# Print the first 5 vectors of the tfidf training data
print(tfidf_train.A[:5])

['00' '000' '0000' '00000031' '000035' '00006' '0001' '0001pt' '000ft'
 '000km']
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


Het creeren van DataFrames om te zien hoe de (woord)-vectoren eruit zien.

In [15]:
# Create the CountVectorizer DataFrame: count_df
count_df = pd.DataFrame(count_train.A, columns=count_vectorizer.get_feature_names_out())

# Create the TfidfVectorizer DataFrame: tfidf_df
tfidf_df = pd.DataFrame(tfidf_train.A, columns=tfidf_vectorizer.get_feature_names_out())

# Print the head of count_df
print(count_df.head())

# Print the head of tfidf_df
print(tfidf_df.head())

# Calculate the difference in columns: difference
difference = set(count_df.columns) - set(tfidf_df.columns)
print(difference)

# Check whether the DataFrames are equal
print(count_df.equals(tfidf_df))

   00  000  0000  00000031  000035  00006  0001  0001pt  000ft  000km  ...  \
0   0    0     0         0       0      0     0       0      0      0  ...   
1   0    0     0         0       0      0     0       0      0      0  ...   
2   0    0     0         0       0      0     0       0      0      0  ...   
3   0    0     0         0       0      0     0       0      0      0  ...   
4   0    0     0         0       0      0     0       0      0      0  ...   

   حلب  عربي  عن  لم  ما  محاولات  من  هذا  والمرضى  ยงade  
0    0     0   0   0   0        0   0    0        0      0  
1    0     0   0   0   0        0   0    0        0      0  
2    0     0   0   0   0        0   0    0        0      0  
3    0     0   0   0   0        0   0    0        0      0  
4    0     0   0   0   0        0   0    0        0      0  

[5 rows x 56922 columns]
    00  000  0000  00000031  000035  00006  0001  0001pt  000ft  000km  ...  \
0  0.0  0.0   0.0       0.0     0.0    0.0   0.0     0.0    

# Naive Bayes Classifer from Scikit Learn

In [19]:
from sklearn.naive_bayes import MultinomialNB
from sklearn import metrics

nb_classifer = MultinomialNB() #creeren een model of classifer

In [37]:
nb_classifer.fit(count_train, y_train) # trainen de classifer/model met de train data.
pred = nb_classifer.predict(count_test) # Voorspellen op basis van het getainde classifer/model wat het resultaat is van de test data.

In [38]:
metrics.accuracy_score(y_test, pred) # We kijken wat de accuraatheid is van de voorspellen met de test labels/data.

0.893352462936394

# Confusion Matrix

In [34]:
metrics.confusion_matrix(y_test, pred, labels=["FAKE","REAL"]) # Om te bepalen hoeveel goede en foute classificaties zijn gedaan.
                                                               # 865 goed, 143 fout "Fake news" labelling
                                                               # 80 fout, 1003 goed "Real news" labelling

array([[ 865,  143],
       [  80, 1003]])

# The Same with TfidfVectorizer

In [39]:
# Import TfidfVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

# Initialize a TfidfVectorizer object: tfidf_vectorizer
tfidf_vectorizer = TfidfVectorizer(stop_words='english', max_df=0.7)

# Transform the training data: tfidf_train 
tfidf_train = tfidf_vectorizer.fit_transform(X_train.values)

# Transform the test data: tfidf_test 
tfidf_test = tfidf_vectorizer.transform(X_test.values)

# Print the first 10 features
print(tfidf_vectorizer.get_feature_names_out()[:10])

# Print the first 5 vectors of the tfidf training data
print(tfidf_train.A[:5])

['00' '000' '0000' '00000031' '000035' '00006' '0001' '0001pt' '000ft'
 '000km']
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [40]:
nb_classifer.fit(tfidf_train, y_train) # trainen de classifer/model met de train data.
pred = nb_classifer.predict(tfidf_test)

In [41]:
metrics.accuracy_score(y_test, pred) # Zoals je ziet een minder goede voorspellingsgraad ivm NB

0.8565279770444764

In [42]:
metrics.confusion_matrix(y_test, pred, labels=["FAKE","REAL"]) # Meer fouten bij het bepalen van 'Fake news". 

array([[ 739,  269],
       [  31, 1052]])